In [9]:
import os
import gc
import warnings
import openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as  sns
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings(action= 'ignore')

In [11]:
train_X = pd.read_csv("./train_origin.csv")
test_X = pd.read_csv("./test_origin.csv")

In [12]:
train_y = train_X['isDefault']
train_X.drop(labels = ['isDefault'], axis = 1, inplace = True)

In [13]:
"""暴力 组合重要特则"""
importance_feature_list = ['grade','subGrade','term','homeOwnership'
,'issueDate_year','n11','n2','employmentLength','ficoRangeLow','n14',
'loanAmnt','dti','n9','issueDate_month','annualIncome','verificationStatus',
'revolBal',
]
temp_list = [] #暂存组合的特征名
result = []
index = 0
temp_train_df = pd.DataFrame()
temp_test_df = pd.DataFrame()
for i in importance_feature_list:
    for j in train_X.columns:
        col = i + '_' + j
        if j + '_' + i in temp_list: #避免重复

            continue
        temp_list.append(col)
        corr =  pearsonr(train_X[i] * train_X[j],train_y)
        if corr[1] <= 0.05 and corr[0] > 0.19:

            result.append([col,corr[0],corr[1]]) #存入列表
            temp_train_df[col] = train_X[i] * train_X[j]
            temp_test_df[col] = test_X[i] * test_X[j]
        del col, corr
        gc.collect()



train_X = pd.concat([train_X, temp_train_df], axis = 1)
test_X = pd.concat([test_X, temp_test_df], axis = 1)

del temp_test_df, temp_train_df
gc.collect()


0

In [14]:
"""四则运算 构造新特征"""
train_X['loanAmnt_annualIncome'] = train_X.apply(lambda x: x['loanAmnt'] / x['annualIncome'] if x['annualIncome'] != 0 else -1, axis = 1)
train_X['installment_loanAmnt'] = train_X.apply(lambda x: x['installment'] / x['loanAmnt'] if x['loanAmnt'] != 0 else -1, axis = 1)
train_X['issueDate_year_earliesCreditLine_year'] = train_X['issueDate_year'] - train_X['earliesCreditLine_year']
train_X['loanAmnt_totalAcc'] = train_X.apply(lambda x: x['loanAmnt'] / x['totalAcc'] if x['totalAcc'] != 0 else -1, axis = 1)
train_X['openAcc_totalAcc'] = train_X.apply(lambda x: x['openAcc'] / x['totalAcc'] if x['totalAcc'] != 0 else -1, axis = 1)
train_X['purpose_totalAcc'] = train_X['purpose'] * train_X['totalAcc']
train_X['purpose_totalAcc'] = train_X['purpose_totalAcc'].apply(lambda x: -1 if x == np.inf else x)

test_X['loanAmnt_annualIncome'] = test_X.apply(lambda x: x['loanAmnt'] / x['annualIncome'] if x['annualIncome'] != 0 else -1, axis = 1)
test_X['installment_loanAmnt'] = test_X.apply(lambda x: x['installment'] / x['loanAmnt'] if x['loanAmnt'] != 0 else -1, axis = 1)
test_X['issueDate_year_earliesCreditLine_year'] = test_X['issueDate_year'] - test_X['earliesCreditLine_year']
test_X['loanAmnt_totalAcc'] = test_X.apply(lambda x: x['loanAmnt'] / x['totalAcc'] if x['totalAcc'] != 0 else -1, axis = 1)
test_X['openAcc_totalAcc'] = test_X.apply(lambda x: x['openAcc'] / x['totalAcc'] if x['totalAcc'] != 0 else -1, axis = 1)
test_X['purpose_totalAcc'] = test_X['purpose'] * train_X['totalAcc']
test_X['purpose_totalAcc'] = test_X['purpose_totalAcc'].apply(lambda x: -1 if x == np.inf else x)


In [16]:
train_X.columns

Index(['id', 'loanAmnt', 'term', 'interestRate', 'installment', 'grade',
       'subGrade', 'employmentTitle', 'employmentLength', 'homeOwnership',
       ...
       'subGrade_earliesCreditLine_year', 'term_interestRate',
       'issueDate_year_interestRate', 'ficoRangeLow_interestRate',
       'loanAmnt_annualIncome', 'installment_loanAmnt',
       'issueDate_year_earliesCreditLine_year', 'loanAmnt_totalAcc',
       'openAcc_totalAcc', 'purpose_totalAcc'],
      dtype='object', length=108)

In [17]:
train_X['isDefault'] = train_y
train_X.to_csv("./train_cross.csv", index = False)
test_X.to_csv("./test_cross.csv", index = False)